In [ ]:
#Importing libraries
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np

In [ ]:

# Loading Haar face cascade model
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Loading caffe models for age and gender prediction
ageProto = "age_deploy.prototxt"
ageModel = "age_net.caffemodel"

genderProto = "gender_deploy.prototxt"
genderModel = "gender_net.caffemodel"

ageNet = cv2.dnn.readNet(ageModel, ageProto)
genderNet = cv2.dnn.readNet(genderModel, genderProto)

# Mean values for the model
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)

# Lists for age and gender prediction
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = ['Male', 'Female']

# Looping over camera feed
cap = cv2.VideoCapture(0)

while 1:
    ret, img = cap.read()
    # Converting image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Detecting faces-Edge detection
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    # Looping over the detected faces
    for (x,y,w,h) in faces:
        # Drawing rectangle on the face region
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        # Extracting detected face
        detected_face = img[y:y+h, x:x+w]
        
        #Bilinear image interpolation
        resized_face=cv2.resize(detected_face,(227,227),interpolation=cv2.INTER_AREA)
        cv2.imshow('face',resized_face)
        
        # Preprocessing with blob to input into the network
        detected_face_blob = cv2.dnn.blobFromImage(detected_face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
       
        # Inputting into the network for gender prediction
        genderNet.setInput(detected_face_blob)
        genderPreds = genderNet.forward()
        gender = genderList[genderPreds[0].argmax()]
        
        # Inputting into the network for age prediction
        ageNet.setInput(detected_face_blob)
        agePreds = ageNet.forward()
        age = ageList[agePreds[0].argmax()]
        
        label = "{}{}".format(gender, age)
        
        # Drawing predictions on frame
        cv2.putText(img, label, (x+30, y+30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
        print("Gender", gender, "AGE", age)
    
    cv2.imshow('img', img)
    k = cv2.waitKey(30) & 0xff
    if k == 27:  # Press Esc to stop the video
        break

cap.release()
cv2.destroyAllWindows()
